In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import myconstant as ct
import math

In [9]:
#import datetime
#import time
import json
import os
import collections
from collections import OrderedDict
import pandas as pd

In [10]:
%run ChartManager.ipynb
#%run Chart.ipynb

In [1]:
class LCUDataset:
    
    def __init__(self, root_dir, c=1, l=0, p=0, ext='jpg', print_label=False):

        self.root_dir = root_dir
        self.chart_num = c
        self.line_num = 2 if l>2 else l
        self.part_num = 3 if p>3 else p
        self.ext = ext
        self.c_list = self._chart_list(c, self.line_num, self.part_num)
        
        self.cm_list = self._cm_list(self.c_list)
        
        self.imgdir = os.path.join(root_dir, "images")
        #self.lpath = "{}/labels.json".format(self.root_dir)
        self.lpath = "{}/labels.csv".format(self.root_dir)
        os.makedirs(self.imgdir, exist_ok=True)
        self.print_label = print_label
        
        self.col = ct.COL_LIST
        self.df = pd.DataFrame(columns=self.col) #헤더 추가
        
        self.img_count = __import__("itertools").count()
        self.count = 0
        print("LCU DATASET INITIALIZATION COMPLETE")
        
    def _chart_list(self, c, l, p):
        c_list = []
        step = 3
        per = step
        pa = int(self.chart_num*step/100)

        for i in range(c):
            chart = Chart(line_num=l, part_num=p)
            #cm = ChartManager(chart)
            #cm.draw()
            #print progress
            if (i != 0) and ((i % pa) == 0):
                print("[CHART GEN PROGRESS] : ", str(per)+" %")
                per = per + step
                
            c_list.append(chart)
            #plt.close()   
            
        print("CHART LIST COMPLETE")        
        return c_list
    
    def _cm_list(self, c_list):
        cm_list = []
        for c in c_list:
            cm_list.append(ChartManager(c))
            
        print("CHART MANAGER LIST COMPLETE")
        return cm_list

    def save_datasets(self, show=True):      
        step = 3
        per = step
        pa = int(self.chart_num*step/100)
        
        print("SAVE IMG START") 
        for i, cm in enumerate(self.cm_list):
            self.count = next(self.img_count)
            cm.draw()
            self.save_image(show)
             
            #print progress
            if (i != 0) and ((i % pa) == 0):
                print("[IMG GEN PROGRESS] : ", str(per)+" %")
                per = per + step
            
            self.save_label_df()
            
        print("SAVE IMG COMPLETE") 
        
        
        self.df.to_csv(self.lpath)
        output_shape = (self.chart_num, len(self.col))
        assert self.df.shape == output_shape
        print("SAVE LABEL COMPLETE")
 
    def save_image(self, show):
        count = self.count
 
        fname = str(count)+"."+self.ext
        path = os.path.join(self.imgdir, fname)

        #print("SAVE IMG : ", path)
        #print("save_plot_img() count :", count)

        plt.savefig(path, transparent=False, facecolor=plt.gcf().get_facecolor(), bbox_inches="tight")
        #plt.savefig(path, quality=80, optimize=False, progressive=False)
        
        
        if show is True:
            plt.show()

        plt.close()


    def save_label_df(self):
        count = self.count
        #print("count : ", count)
        label = self._label(count)
        self.df.loc[count+1] = label
        
            
        
    #i th  chart's  label    
    def _label(self, i):
        c = self.c_list[i]
        label = [np.nan for _ in  range(len(ct.COL_LIST))]
        #label = []
        
        # superiority
        label[ct.SUPERIORITY] = c.superiority
        
        
        # partition number
        label[ct.L1_P_NUM] = c.l1p_num
        label[ct.L2_P_NUM] = c.l2p_num

        
        # monotonic
        label[ct.L1_MONOTONIC] = c.l1_mon
        label[ct.L2_MONOTONIC] = c.l2_mon
        
        
        # minmax
        label[ct.L1_MINMAX_1 : ct.L1_MINMAX_2] = c.l1_min + c.l1_max
        label[ct.L2_MINMAX_1 : ct.L2_MINMAX_2] = c.l2_min + c.l2_max
        
        
        # partition start, end
        label[ct.L1_SE_1 : ct.L1_SE_2] = c.l1_se
        label[ct.L2_SE_1 : ct.L2_SE_2] = c.l2_se
        
        
        # growth type
        label[ct.L1_GROWTH_1 : ct.L1_GROWTH_2] = c.l1_g
        label[ct.L2_GROWTH_1 : ct.L2_GROWTH_2] = c.l2_g
        
        
        if self.print_label:
            self._print_label(c, label)
        
        return label
        
    def _print_label(self, c, label):
        print("-"*150)
        print("label length : ", len(label))
        
        print("-"*150)
        print("superiority : ", c.superiority)
        
        print("-"*150)
        print("l1p_num : ", c.l1p_num)
        print("l2p_num : ", c.l2p_num)

        
        print("-"*150)
        print("l1_mon : ", c.l1_mon)
        print("l2_mon : ", c.l2_mon)
        
        print("-"*150)
        #print("l1_min : ", len(c.l1_min))
        #print("l2_min : ", len(c.l2_min))
        print("l1_min : ", c.l1_min)
        print("l2_min : ", c.l2_min)
        print("-"*150)
        #print("l1_max : ", len(c.l1_max))
        #print("l2_max : ", len(c.l2_max))
        print("l1_max : ", c.l1_max)
        print("l2_max : ", c.l2_max)
        
        print("-"*150)
        print("l1_se : ", c.l1_se)
        print("l2_se : ", c.l2_se)
        
        print("-"*150)
        #print("l1r : ", len(c.l1r))
        #print("l2r : ", len(c.l2r))
        print("l1_g : ", c.l1_g)
        print("l2_g : ", c.l2_g)
        